<a href="https://colab.research.google.com/github/vitorhugo28/projects/blob/main/transfer_learning_Cats_and_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
from IPython import get_ipython
from IPython.display import display
%matplotlib inline
# Configura o matplotlib para exibir gráficos diretamente no notebook.
import os # Importa o módulo para interagir com o sistema operacional, como manipular arquivos e diretórios.
import random # Importa o módulo para gerar números aleatórios.
import numpy as np # Importa a biblioteca NumPy para trabalhar com arrays multidimensionais.
import keras # Importa a biblioteca Keras para construir e treinar redes neurais.
import matplotlib.pyplot as plt # Importa o módulo pyplot da biblioteca Matplotlib para criar gráficos.
from matplotlib.pyplot import imshow # Importa a função imshow para exibir imagens.
from keras.preprocessing import image # Importa o módulo de pré-processamento de imagens do Keras.
from keras.applications.imagenet_utils import preprocess_input # Importa a função preprocess_input para pré-processar imagens de acordo com o ImageNet.
from keras.models import Sequential # Importa a classe Sequential para criar modelos de redes neurais sequenciais.
from keras.layers import Dense, Dropout, Flatten, Activation # Importa as camadas Dense (totalmente conectada), Dropout (para evitar overfitting), Flatten (para transformar dados multidimensionais em unidimensionais) e Activation (para aplicar funções de ativação).
from keras.layers import Conv2D, MaxPooling2D # Importa as camadas Conv2D (convolução 2D) e MaxPooling2D (pooling máximo 2D).
from keras.models import Model # Importa a classe Model para criar modelos de redes neurais mais complexos.

In [ ]:
!curl -L -o kagglecatsanddogs_5340.zip --progress-bar https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip # Baixa o arquivo zip do dataset Kaggle Cats and Dogs usando o comando curl.
!unzip kagglecatsanddogs_5340.zip # Descompacta o arquivo zip baixado.
!rm kagglecatsanddogs_5340.zip # Remove o arquivo zip após a descompactação.

############################################################################################# 100.0%
Archive:  kagglecatsanddogs_5340.zip
replace PetImages/Cat/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
root = 'PetImages'  # Define a variável root como o caminho para o diretório raiz do dataset.
train_split, val_split = 0.7, 0.15 # Define as proporções para divisão do dataset em treino e validação.

categories = ['Cat', 'Dog']  # Define as categorias do novo dataset (gato e cachorro).
num_classes = len(categories) # Calcula o número de classes (2).

data = [] # Inicializa uma lista vazia para armazenar os dados das imagens.
for c, category in enumerate(categories): # Itera pelas categorias, enumerando-as.
    images = [os.path.join(root, category, f) for f in os.listdir(os.path.join(root, category))
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']] # Cria uma lista de caminhos para todas as imagens na categoria atual.
    for img_path in images: # Itera pelos caminhos das imagens.
        try:  # Adicione um try-except para lidar com imagens corrompidas
            img, x = get_image(img_path) # Chama a função get_image para carregar a imagem e obter seu vetor de entrada.
            data.append({'x':np.array(x[0]), 'y':c}) # Adiciona um dicionário com o vetor de entrada e a categoria correspondente à lista de dados.
        except:
            pass

In [ ]:
# Função auxiliar para carregar a imagem e retornar a imagem e o vetor de entrada.
def get_image(path):
    img = image.load_img(path, target_size=(224, 224)) # Carrega a imagem do caminho especificado, redimensionando-a para 224x224 pixels.
    x = image.img_to_array(img) # Converte a imagem em um array NumPy.
    x = np.expand_dims(x, axis=0) # Adiciona uma dimensão extra ao array para representar o batch (lote) de imagens.
    x = preprocess_input(x) # Pré-processa a imagem de acordo com o ImageNet.
    return img, x # Retorna a imagem e o vetor de entrada pré-processado.

In [ ]:
data = [] # Inicializa uma lista vazia para armazenar os dados das imagens.
for c, category in enumerate(categories): # Itera pelas categorias, enumerando-as.
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']] # Cria uma lista de caminhos para todas as imagens na categoria atual, incluindo subdiretórios.
    for img_path in images: # Itera pelos caminhos das imagens.
        img, x = get_image(img_path) # Chama a função get_image para carregar a imagem e obter seu vetor de entrada.
        data.append({'x':np.array(x[0]), 'y':c}) # Adiciona um dicionário com o vetor de entrada e a categoria correspondente à lista de dados.

model = Sequential()  # Define o modelo como uma rede neural sequencial.

model.add(Dense(num_classes))  # Adiciona uma camada densa (totalmente conectada) com o número de neurônios igual ao número de classes (2).
model.add(Activation('softmax')) # Adiciona uma camada de ativação softmax para obter probabilidades para cada classe.


In [ ]:
random.shuffle(data) # Embaralha aleatoriamente os dados das imagens.

In [ ]:
idx_val = int(train_split * len(data)) # Calcula o índice para dividir o dataset em treino e validação.
idx_test = int((train_split + val_split) * len(data)) # Calcula o índice para dividir o dataset em validação e teste.
train = data[:idx_val] # Cria a lista de dados de treino.
val = data[idx_val:idx_test] # Cria a lista de dados de validação.
test = data[idx_test:] # Cria a lista de dados de teste.

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train] # Separa os vetores de entrada (x_train) e as categorias (y_train) dos dados de treino.
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val] # Separa os vetores de entrada (x_val) e as categorias (y_val) dos dados de validação.
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test] # Separa os vetores de entrada (x_test) e as categorias (y_test) dos dados de teste.
print(y_test) # Imprime as categorias dos dados de teste.

In [ ]:
# Normaliza os dados dividindo os valores dos pixels por 255.
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Converte as categorias em vetores one-hot (codificação one-hot).
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape) # Imprime o formato dos vetores one-hot dos dados de teste.

In [ ]:
# Imprime um resumo dos dados carregados.
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)


In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']] # Cria uma lista de caminhos para todas as imagens no diretório raiz, incluindo subdiretórios.
idx = [int(len(images) * random.random()) for i in range(8)] # Seleciona aleatoriamente 8 índices de imagens.
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx] # Carrega as 8 imagens selecionadas, redimensionando-as para 224x224 pixels.
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1) # Concatena as imagens horizontalmente.
plt.figure(figsize=(16,4)) # Cria uma figura para exibir as imagens.
plt.imshow(concat_image) # Exibe as imagens concatenadas.

In [ ]:
# Constrói a rede neural.
model = Sequential() # Define o modelo como uma rede neural sequencial.
print("Input dimensions: ",x_train.shape[1:]) # Imprime as dimensões de entrada da rede neural.

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:])) # Adiciona uma camada de convolução 2D com 32 filtros de tamanho 3x3, usando as dimensões de entrada especificadas.
model.add(Activation('relu')) # Adiciona uma camada de ativação ReLU (Unidade Linear Retificada).
model.add(MaxPooling2D(pool_size=(2, 2))) # Adiciona uma camada de pooling máximo 2D com tamanho de pool 2x2.

model.add(Conv2D(32, (3, 3))) # Adiciona outra camada de convolução 2D com 32 filtros de tamanho 3x3.
model.add(Activation('relu')) # Adiciona uma camada de ativação ReLU.
model.add(MaxPooling2D(pool_size=(2, 2))) # Adiciona uma camada de pooling máximo 2D.

model.add(Dropout(0.25)) # Adiciona uma camada de dropout com taxa de 0.25 para evitar overfitting.

model.add(Conv2D(32, (3, 3))) # Adiciona outra camada de convolução 2D.
model.add(Activation('relu')) # Adiciona uma camada de ativação ReLU.
model.add(MaxPooling2D(pool_size=(2, 2))) # Adiciona uma camada de pooling máximo 2D.

model.add(Conv2D(32, (3, 3))) # Adiciona outra camada de convolução 2D.
model.add(Activation('relu')) # Adiciona uma camada de ativação ReLU.
model.add(MaxPooling2D(pool_size=(2, 2))) # Adiciona uma camada de pooling máximo 2D.

model.add(Dropout(0.25)) # Adiciona uma camada de dropout.

model.add(Flatten()) # Adiciona uma camada de flatten para transformar os dados multidimensionais em unidimensionais.
model.add(Dense(256)) # Adiciona uma camada densa com 256 neurônios.
model.add(Activation('relu')) # Adiciona uma camada de ativação ReLU.

model.add(Dropout(0.5)) # Adiciona uma camada de dropout com taxa de 0.5.

model.add(Dense(num_classes)) # Adiciona uma camada densa com o número de neurônios igual ao número de classes.
model.add(Activation('softmax')) # Adiciona uma camada de ativação softmax.

model.summary() # Imprime um resumo da arquitetura da rede neural.

In [ ]:
# Compila o modelo usando a função de perda categorical cross-entropy, o otimizador Adam e a métrica de acurácia.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Treina o modelo usando os dados de treino, com tamanho de batch 128 e 10 épocas, e valida o modelo a cada época usando os dados de validação.
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))


In [ ]:
# Cria um gráfico com a perda de validação e a acurácia de validação ao longo das épocas.
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show() # Exibe o gráfico.

In [ ]:
# Avalia o modelo usando os dados de teste e imprime a perda e a acurácia do teste.
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)

In [ ]:
# Carrega o modelo VGG16 pré-treinado no ImageNet.
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary() # Imprime um resumo da arquitetura do modelo VGG16.

In [ ]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)


In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

In [ ]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))


In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.plot(history2.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
img, x = get_image('101_ObjectCategories/airplanes/image_0003.jpg')
probabilities = model_new.predict([x])
